In [6]:
!pip install dask
!pip install pyarrow
!pip install numpy
!pip install pandas
!pip install polars


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.6/34.6 MB 35.0 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [7]:
import dask.dataframe as dd
import polars as pl
import numpy as np
import os
import pandas as pd

### Wygenerowanie duzego pliku

In [11]:
rows_per_chunk = 1_000_000
total_rows = 100_000_000  # ok. 10GB
filename = "bigfile_polars.csv"
categories = np.array(['A', 'B', 'C', 'D'])
np.random.seed(42)

# Usuń plik, jeśli istnieje
if os.path.exists(filename):
    os.remove(filename)

for i, start in enumerate(range(0, total_rows, rows_per_chunk)):
    end = min(start + rows_per_chunk, total_rows)
    n_rows = end - start
    data = {
        "value": np.random.randint(0, 10_000, size=n_rows),
        "category": np.random.choice(categories, size=n_rows)
    }
    df = pl.DataFrame(data)
    if i == 0:
        df.write_csv(filename)
    else:
        with open(filename, "a") as f:
            df.write_csv(f, include_header=False)
    print(f"Zapisano wiersze {start} - {end}")

print("Zrobione!")

Zapisano wiersze 0 - 1000000
Zapisano wiersze 1000000 - 2000000
Zapisano wiersze 2000000 - 3000000
Zapisano wiersze 3000000 - 4000000
Zapisano wiersze 4000000 - 5000000
Zapisano wiersze 5000000 - 6000000
Zapisano wiersze 6000000 - 7000000
Zapisano wiersze 7000000 - 8000000
Zapisano wiersze 8000000 - 9000000
Zapisano wiersze 9000000 - 10000000
Zapisano wiersze 10000000 - 11000000
Zapisano wiersze 11000000 - 12000000
Zapisano wiersze 12000000 - 13000000
Zapisano wiersze 13000000 - 14000000
Zapisano wiersze 14000000 - 15000000
Zapisano wiersze 15000000 - 16000000
Zapisano wiersze 16000000 - 17000000
Zapisano wiersze 17000000 - 18000000
Zapisano wiersze 18000000 - 19000000
Zapisano wiersze 19000000 - 20000000
Zapisano wiersze 20000000 - 21000000
Zapisano wiersze 21000000 - 22000000
Zapisano wiersze 22000000 - 23000000
Zapisano wiersze 23000000 - 24000000
Zapisano wiersze 24000000 - 25000000
Zapisano wiersze 25000000 - 26000000
Zapisano wiersze 26000000 - 27000000
Zapisano wiersze 27000000 

## Sprawdzenie różnic wydajności

In [14]:
# Pandas
import time
import dask.dataframe as dd
import time
import polars as pl
import time

import pandas as pd
import time

filename = "bigfile_polars.csv"
#############
start = time.time()
df = pd.read_csv(filename)
mean = df["value"].mean()
print("Pandas mean:", mean)
print("Czas:", time.time() - start)

start = time.time()
result = df.groupby("category")["value"].sum()
print("Pandas groupby sum:", result.head())
print("Czas:", time.time() - start)

###############
# polars
start = time.time()
df = pl.read_csv(filename)
mean = df["value"].mean()
print("Polars mean:", mean)
print("Czas:", time.time() - start)

# Polars
start = time.time()
result = df.group_by("category").agg(pl.col("value").sum())
print("Polars groupby sum:", result.head())
print("Czas:", time.time() - start)

start = time.time()

# Lazy scan (nie ładuje całego pliku do RAM)
df = pl.scan_csv(filename)

# Lazy pipeline, wynik będzie policzony podczas collect()
mean = df.select(pl.col("value").mean()).collect()
print("Polars scan mean:", mean)

print("Czas:", time.time() - start)
# Polars
start = time.time()
result = (
    pl.scan_csv(filename)
    .group_by("category")
    .agg(pl.col("value").sum())
    .collect()   # <-- TO JEST KLUCZ!
)
# print("Polars groupby sum:", result.head())
print("Polars groupby sum:")
print("Czas:", time.time() - start)

####################
# dask
start = time.time()
df = dd.read_csv(filename)
mean = df["value"].mean().compute()
print("Dask mean:", mean)
print("Czas:", time.time() - start)


# Dask
start = time.time()
result = df.groupby("category")["value"].sum().compute()
# print("Dask groupby sum:", result.head())
print("Dask groupby sum:")
print("Czas:", time.time() - start)


Pandas mean: 5000.35136653
Czas: 6.925375938415527
Pandas groupby sum: category
A    125019660990
B    125004971437
C    125014117338
D    124996386888
Name: value, dtype: int64
Czas: 2.5247740745544434
Polars mean: 5000.35136653
Czas: 1.2233951091766357
Polars groupby sum: shape: (4, 2)
┌──────────┬──────────────┐
│ category ┆ value        │
│ ---      ┆ ---          │
│ str      ┆ i64          │
╞══════════╪══════════════╡
│ B        ┆ 125004971437 │
│ A        ┆ 125019660990 │
│ D        ┆ 124996386888 │
│ C        ┆ 125014117338 │
└──────────┴──────────────┘
Czas: 1.9061551094055176
Polars scan mean: shape: (1, 1)
┌─────────────┐
│ value       │
│ ---         │
│ f64         │
╞═════════════╡
│ 5000.351367 │
└─────────────┘
Czas: 0.6150121688842773
Polars groupby sum:
Czas: 3.114590883255005
Dask mean: 5000.35136653
Czas: 4.7341628074646
Dask groupby sum:
Czas: 9.653946161270142


In [13]:
# filter
import time
import pandas as pd
import polars as pl

filename = "bigfile_polars.csv"

# ------ POLARS (read) ------
start = time.time()
df_polars = pl.read_csv(filename)
filtered_polars = df_polars.filter(pl.col("category") == "B")
print("Polars: liczba wierszy z kategorią 'B':", filtered_polars.height)
print("Polars filter czas:", time.time() - start)

# ------ POLARS (scan, czyli lazy – nie trzyma całego pliku w RAM) ------
import time
import polars as pl
filename = "bigfile_polars.csv"
start = time.time()
df_polars = pl.scan_csv(filename)
filtered_polars = df_polars.filter(pl.col("category") == "B").collect()
print("Polars: liczba wierszy z kategorią 'B':", filtered_polars.height)
print("Polars filter czas:", time.time() - start)

# ------ PANDAS (uwaga: ładuje cały plik do RAM!) ------
import time
import pandas as pd
filename = "bigfile_polars.csv"
start = time.time()
df_pandas = pd.read_csv(filename)
filtered_pandas = df_pandas[df_pandas["category"] == "B"]
print("Pandas: liczba wierszy z kategorią 'B':", len(filtered_pandas))
print("Pandas filter czas:", time.time() - start)

###############
# dask
import dask.dataframe as dd
import time

df = dd.read_csv("bigfile_polars.csv")
filtered_dask = df[df["category"] == "B"]

start = time.time()
result = filtered_dask.shape[0].compute()
end = time.time()

print("Dask: liczba wierszy z kategorią 'B':", result)
print("Czas:", end - start)

Polars: liczba wierszy z kategorią 'B': 24997100
Polars filter czas: 2.465214967727661
Polars: liczba wierszy z kategorią 'B': 24997100
Polars filter czas: 1.1024436950683594
Pandas: liczba wierszy z kategorią 'B': 24997100
Pandas filter czas: 10.04599118232727
Dask: liczba wierszy z kategorią 'B': 24997100
Czas: 11.55773377418518
